In [1]:
import os
from pathlib import Path
from sqlalchemy import create_engine

import xmlrpc.client
import pandas as pd
pd.options.display.float_format = '{:,.2f}'.format

api_url = os.environ.get('ODOO_URL_API')
api_db = os.environ.get('ODOO_DB_API')
api_username = os.environ.get('ODOO_USERNAME_API')
api_clave = os.environ.get('ODOO_CLAVE_API')

common = xmlrpc.client.ServerProxy(f'{api_url}/xmlrpc/2/common')
uid = common.authenticate(api_db, api_username, api_clave, {})
models = xmlrpc.client.ServerProxy(f'{api_url}/xmlrpc/2/object')

db_file = 'comisiones.db'
db_file_path_str = str(Path().cwd().parent.parent.joinpath(f'data/{db_file}'))

engine = create_engine(f'sqlite:///{db_file_path_str}')

In [2]:
with engine.connect() as conn, conn.begin():  
    ventas_enero = pd.read_sql_table('ventas_enero', conn, dtype_backend='numpy_nullable')
    ventas_febrero = pd.read_sql_table('ventas_febrero', conn, dtype_backend='numpy_nullable')
    ventas_marzo = pd.read_sql_table('ventas_marzo', conn, dtype_backend='numpy_nullable')
    ventas_abril = pd.read_sql_table('ventas_abril', conn, dtype_backend='numpy_nullable')
    ventas_mayo = pd.read_sql_table('ventas_mayo', conn, dtype_backend='numpy_nullable')
    ventas_junio = pd.read_sql_table('ventas_junio', conn, dtype_backend='numpy_nullable')
    ventas_año = pd.read_sql_table('ventas_año', conn, dtype_backend='numpy_nullable')

engine.dispose()

In [3]:
db_file1 = 'proveedores_oficiales.xlsx'
db_file1_path_str = str(Path().cwd().parent.parent.joinpath(f'data/compras/{db_file1}'))

proveedores = pd.read_excel(db_file1_path_str)
prov_oficiales = proveedores.loc[proveedores['oficial'] == 1][['partner_id', 'partner_name']]
prov_locales = proveedores.loc[proveedores['oficial'] == 0][['partner_id', 'partner_name']]

In [4]:
db_file2 = 'productos_sin_compra.xlsx'
db_file2_path_str = str(Path().cwd().parent.parent.joinpath(f'data/compras/{db_file2}'))

productos_sin_compra = pd.read_excel(db_file2_path_str)

In [5]:
fields_compras_doc = ['name', 'state','partner_id', 'partner_ref', 'date_approve', 'x_fecha_factura', 'user_id', 'create_uid']

ids_compras_doc = models.execute_kw(api_db, uid, api_clave, 'purchase.order', 'search', [[("state", "in", ("purchase", "done"))]])
json_compras_doc = models.execute_kw(api_db, uid, api_clave, 'purchase.order', 'read', [ids_compras_doc], {'fields': fields_compras_doc})

In [6]:
data_compras_doc = []

for compra in json_compras_doc:
    new = {}
    new['order_id'] = compra['id']
    new['order_name'] = compra['name']
    new['order_state'] = compra['state']
    new['order_date'] = compra['date_approve'] if compra['date_approve'] else pd.NA
    new['partner_id'] = compra['partner_id'][0]
    new['partner_name'] = compra['partner_id'][1]
    new['partner_fact_ref'] = compra['partner_ref']
    new['partner_fact_date'] = compra['x_fecha_factura'] if compra['x_fecha_factura'] else pd.NA
    new['capturista'] = compra['create_uid'][1] if compra['create_uid'] else pd.NA
    new['vendedora'] = compra['user_id'][1] if compra['user_id'] else pd.NA

    data_compras_doc.append(new)

compras_doc = pd.DataFrame(data_compras_doc)
compras_doc['order_date'] = pd.to_datetime(compras_doc['order_date'], format='%Y-%m-%d %H:%M:%S')
compras_doc['partner_fact_date'] = pd.to_datetime(compras_doc['partner_fact_date'], format='%Y-%m-%d')

In [7]:
fields_compras_line = ['order_id', 'date_approve', 'partner_id','product_id', 'product_qty', 'price_unit_discounted']

ids_compras_line = models.execute_kw(api_db, uid, api_clave, 'purchase.order.line', 'search', [[("order_id.id", "in", ids_compras_doc)]])
json_compras_line = models.execute_kw(api_db, uid, api_clave, 'purchase.order.line', 'read', [ids_compras_line], {'fields': fields_compras_line})

In [8]:
data_compras_line = []

for line in json_compras_line:
    new = {}
    new['line_id'] = line['id']
    new['order_id'] = line['order_id'][0]
    new['order_name'] = line['order_id'][1]
    new['order_date'] = line['date_approve'] if line['date_approve'] else pd.NA
    new['partner_id'] = line['partner_id'][0]
    new['partner_name'] = line['partner_id'][1]
    new['product_id_pp'] = line['product_id'][0]
    new['product_name'] = line['product_id'][1]
    new['product_qty'] = line['product_qty']
    new['product_price'] = line['price_unit_discounted']
    
    data_compras_line.append(new)

compras_linea = pd.DataFrame(data_compras_line)
compras_linea['order_date'] = pd.to_datetime(compras_linea['order_date'], format='%Y-%m-%d %H:%M:%S')

compras_linea['oficial'] = compras_linea['partner_id'].isin(prov_oficiales['partner_id'])

In [9]:
compras = pd.merge(compras_linea,
                   compras_doc[['order_id', 'partner_fact_ref', 'partner_fact_date', 'capturista', 'vendedora']], 
                    how='left', 
                    on='order_id')

In [10]:
# Línea para comprobrar que el 100% de los proveedores de Odoo están calificados en la lista de proveedores oficiales

check1 = (compras_doc[~compras_doc['partner_id'].isin(proveedores['partner_id'])]).drop_duplicates('partner_id')
not check1.empty and print('Hay proveedores no calificados')
check1


,order_id,order_name,order_state,order_date,partner_id,partner_name,partner_fact_ref,partner_fact_date,capturista,vendedora


# Pruebas

In [11]:
ventas_año['costo_producto'] = pd.NA
ventas_año['costo_order_date'] = pd.NaT
ventas_año['costo_order_line_id'] = pd.NA

for i in range(len(ventas_año)):

    minidf = (compras[
            (compras['order_date'] <= pd.to_datetime(ventas_año['invoice_date'].iloc[i]) + pd.Timedelta(days=5)) 
            & (compras['product_id_pp'] == ventas_año['product_id'].iloc[i])]
            .sort_values('order_date', ascending = False)
            )

    if not minidf.empty:
        ventas_año.iloc[i,30] = minidf['product_price'].iloc[0]
        ventas_año.iloc[i,31] = minidf['order_date'].iloc[0]
        ventas_año.iloc[i,32] = minidf['line_id'].iloc[0]

    else:
        minidf_costo_inicial = productos_sin_compra[productos_sin_compra['product_id'] == ventas_año['product_id'].iloc[i]]

        if not minidf_costo_inicial.empty:
            ventas_año.iloc[i,30] = minidf_costo_inicial['costo_producto'].iloc[0]
            ventas_año.iloc[i,31] = minidf_costo_inicial['costo_sae_date'].iloc[0]

In [12]:
# ventas_año[ventas_año['costo_producto'].isna()].drop_duplicates('product_id').to_excel('wep.xlsx')
ventas_año[ventas_año['costo_producto'].isna()].drop_duplicates('product_id')[['product_id', 'product_name']]

,product_id,product_name
10166,14757,Nuevo *0*
10641,28586,Anticipo
13243,28638,Servicios de Facturación
18271,4,Anticipo (PdV)


In [13]:
#Borrar esta línea, es provisional. Se tienen que corregir usando pd.NaN

for i in range(len(ventas_año['price_subtotal'])):
    ventas_año['price_subtotal'].iloc[i] = 0.01 if ventas_año['price_subtotal'].iloc[i] == 0 else ventas_año['price_subtotal'].iloc[i]

C:\Users\Lic.Costich\AppData\Local\Temp\ipykernel_5676\233693957.py:4: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  ventas_año['price_subtotal'].iloc[i] = 0.01 if ventas_año['price_subtotal'].iloc[i] == 0 else ventas_año['price_subtotal'].i

In [14]:
ventas_año['costo_date_dif'] = ((ventas_año['invoice_date'] - ventas_año['costo_order_date']).dt.days).astype('Int64')
ventas_año['costo_subtotal'] = ventas_año['quantity'] * ventas_año['costo_producto']
ventas_año['utilidad_subtotal'] = ventas_año['price_subtotal'] - ventas_año['costo_subtotal']
ventas_año['utilidad_%'] = ((ventas_año['price_subtotal'] / ventas_año['costo_subtotal']) - 1) * 100
ventas_año['marg_util_%'] = (ventas_año['utilidad_subtotal'] / ventas_año['price_subtotal']) * 100


cols_ventas = ['fact_doc_id', 'name', 'invoice_date', 'partner_id',
       'partner_name', 'salesperson_id', 'salesperson_name', 'sale_team_description', 'business_model',
       'product_id', 'product_name', 'quantity', 'price_subtotal',
       'costo_subtotal', 'costo_order_date', 'costo_order_line_id',
       'costo_date_dif', 'utilidad_subtotal', 'utilidad_%', 'marg_util_%', 'costo_producto']

ventas = ventas_año[cols_ventas]

In [15]:
wep = compras[['partner_id', 'partner_name', 'product_id_pp', 'product_name']].groupby('product_id_pp').agg({'product_name': ['first'], 'partner_id': ['mean', 'first']})
wep.columns = ['product_name', 'mean', 'first']
wep['wep'] = wep['mean']  == wep['first']
wep.loc[wep['wep'] == False]

,product_name,mean,first,wep
product_id_pp,,,,
7480,"[2538] Rondana Plana 1/4"" C#181 *2538*","5,337.00",5366,False
7481,[2540] Rondana Plana 3/16 C#184 *2540*,"5,293.50",5366,False
7483,"[6528] Pija Autorr. C/Rondana 8""*1/2"" Galv. *6...","5,345.29",5366,False
7510,"[5388] Tuerca Hexagonal 1/4"" Hncz (5600/Ct) C...","5,329.75",5366,False
7511,"[10179] Taq. Plastico El-Pro 1/4"" Blanco TQ-01...","5,328.20",5313,False
...,...,...,...,...
29236,[4413] Brida Flexible Coflex PB-300 Larga *4413*,"5,326.33",5335,False
29241,[4426] Color Cemento Negro *4426*,"5,236.50",5143,False
29255,[4497] Guantes De Corte De Nitrilo 48-22-8902 ...,"5,179.80",5128,False


In [16]:
wep.loc[wep['wep'] == False].reset_index()['product_id_pp']

0       7480
1       7481
2       7483
3       7510
4       7511
       ...  
203    29236
204    29241
205    29255
206    29436
207    29452
Name: product_id_pp, Length: 208, dtype: int64

In [17]:
compras.loc[compras['product_id_pp'].isin(wep.loc[wep['wep'] == False].reset_index()['product_id_pp'])]

,line_id,order_id,order_name,order_date,partner_id,partner_name,product_id_pp,product_name,product_qty,product_price,oficial,partner_fact_ref,partner_fact_date,capturista,vendedora
5,18959,3093,P03079 (MXF 759368),2024-06-24 21:42:34,5244,Makita Mexico,14423,"[MAK9554HNG] Esmeriladora Ang 4 1/2"" 710W 9554...",5.00,943.00,True,MXF 759368,2024-06-20,Alexa Yadira Mazariegos Zunun,Mariana Araceli Carvajal Flores
72,18837,3089,P03075 (AA 1072328),2024-06-24 21:08:30,5283,Chapa Industrias,9169,[13374] Bis. Bidi. Brazo Corto #3655 (Par) *13...,250.00,19.25,True,AA 1072328,2024-06-24,Alexa Yadira Mazariegos Zunun,Rosario Martinez Zarate
79,18844,3089,P03075 (AA 1072328),2024-06-24 21:08:30,5283,Chapa Industrias,11326,"[15112] Pija Drywall 8 x 2"" 15422 (Kg) *15112*",50.00,70.63,True,AA 1072328,2024-06-24,Alexa Yadira Mazariegos Zunun,Rosario Martinez Zarate
101,18777,3082,P03068 (SJCFCO 16222),2024-06-24 15:43:49,5326,Madereria El Pino De Los Cabos,12827,[13712] Barrote 2 * 4 * 12 Americano *13712*,25.00,181.04,False,SJCFCO 16222,2024-06-22,Alexa Yadira Mazariegos Zunun,Yamilet Blanco Salas
102,18776,3082,P03068 (SJCFCO 16222),2024-06-24 15:43:49,5326,Madereria El Pino De Los Cabos,14548,[24622] Tabla Teka Nacional 1x8 (Cama) *24622*,25.00,"1,862.07",False,SJCFCO 16222,2024-06-22,Alexa Yadira Mazariegos Zunun,Yamilet Blanco Salas
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12533,61,47,P00040 (4HGFFI 172807),2024-01-04 00:02:18,5387,Home Depot Mexico,14371,"[19897] Lijadora orb. 5"" Dewalt Dwe6421-B3 *19...",2.00,"1,525.00",False,4HGFFI 172807,NaT,<NA>,Mayra Angelica Parada Manjarrez
12540,54,40,P00033 (2000005232684875),2024-01-03 21:34:27,5128,Mercado Libre,14773,[1375] Disco Abrasivo Desbaste Metal 4-1/2x7/8...,25.00,25.14,False,2000005232684875,NaT,<NA>,Yolanda Alejandra Rodriguez González
12589,5,2,P00002 (SJCFCO 23),2024-01-03 00:09:24,5326,Madereria El Pino De Los Cabos,13371,[12749] Triplay Pino 1/4 4'*8' 1/Cara *12749*,10.00,267.25,False,SJCFCO 23,NaT,<NA>,Yamilet Blanco Salas
12591,2,1,P00001 (SJCFCO 68),2024-01-03 00:05:53,5326,Madereria El Pino De Los Cabos,14272,[13508] Tablon Americano 2x12x12 *13508*,2.00,910.34,False,SJCFCO 68,NaT,<NA>,Yamilet Blanco Salas


In [18]:
wep2 = compras.loc[compras['product_id_pp'].isin(wep.loc[wep['wep'] == False].reset_index()['product_id_pp'])]
# wep2.loc[(wep2['oficial'] == False) & (wep2['product_id_pp'] == 12827)].to_excel('c.xlsx')

In [19]:
# ventas.loc[ventas['product_id'] == 12827].to_excel('v.xlsx')